# Transformada Rápida de Fourier e a sua Inversa em JuliaLang
---
# Sumário
<html>
<body>
<ul>
    <li><a href="#1.-Transformada-Rápida-de-Fourier-(FFT)">1. Transformada Rápida de Fourier (FFT)</a></li>
</ul>
<ul>
    <li><a href="#2.-Inversa-da-Transformada-Rápida-de-Fourier-(IFFT)">2. Inversa da Transformada Rápida de Fourier (IFFT)</a></li>
</ul>
<ul>
    <li><a href="#3.-Bibliotecas-do-Julia-para-FFT">3. Bibliotecas do Julia para FFT</a></li>
        <ul>
            <li><a href="#3.1-CLFFT">3.1 CLFFT</a></li>
            <li><a href="#3.2-NFFT">3.2 NFFT</a></li>
        </ul>
</ul>
<ul>
    <li><a href="#4.-Conclusão">4. Conclusão</a></li>
</ul>
<ul>
    <li><a href="#5.-Referências">5. Referências</a></li>
</ul>

</body>
</html>         

## 1. Transformada Rápida de Fourier (FFT)

- Criando MyFFT :

In [1]:
function myFFT(x)
    # Input : x - vetores de entrada
    # Output : X - coeficientes de fourier
    N = length(x)
    #Inicializando os coeficientes de fourier da saida
    X = zeros(1,N) + 1im*zeros(1,N)
    #Fator de ponderação
    W = exp(-1im*2*pi/N)
    
    if N == 2
        X[1] = x[1] + x[2]
        X[2] = x[1] - x[2]
    else
        for k = 1:N/2
            Pares = myFFT(x[1:2:end])
            Impares = myFFT(x[2:2:end])
            X[k] = Pares[k] + W^(k-1)*Impares[k]
            X[k+N/2] = Pares[k] + W^(k-1+N/2)*Impares[k]
        end
    end
    return X'
end

myFFT (generic function with 1 method)

In [2]:
#N = 4096
N = 32
vectorTeste_fft = Complex32[1:1:N];
getTime_myFFT = @time myFFT(vectorTeste_fft);

elapsed time: 0.246844618 seconds (19160584 bytes allocated)


- Utilizando a função Nativa do Julia :

In [3]:
#N = 4096
N = 32
vector = Complex32[1:1:N];
getTime_ntFFT = @time fft(vector);

elapsed time: 0.195218919 seconds (6178220 bytes allocated)


## 2. Inversa da Transformada Rápida de Fourier (IFFT)

- Criando MyIFFT :

In [4]:
function myIFFT(x)
    N = length(x)
    return (1/N)*myFFT(x)
end

myIFFT (generic function with 1 method)

In [5]:
#N = 4096
N = 32
vectorTeste_ifft = Complex32[1:1:N];
getTime_myIFFT = @time myIFFT(vectorTeste_ifft);

elapsed time: 0.034236323 seconds (11937700 bytes allocated)


- Utilizando a função Nativa do Julia :

In [12]:
N = 4096
vector_IFFT = [1:1:N];
getTime_ntIFFT = @time ifft(vector_IFFT);

elapsed time: 0.006755123 seconds (67512 bytes allocated)


## 3. Bibliotecas do Julia para FFT

### 3.1 CLFFT

A CLFFT fornece um conjunto de rotinas de FFT que são otimizados para os processadores gráficos da AMD, e que também são funcionais em toda a CPU e outros dispositivos de computação.

In [15]:
import OpenCL
import CLFFT

const cl = OpenCL
const clfft = CLFFT

_, ctx, queue = cl.create_compute_context()

function myCLFFT(x)
    N = length(x)
    X = ones(Complex64, N)
    bufX = cl.Buffer(Complex64, ctx, :copy, hostbuf=X)

    p = clfft.Plan(Complex64, ctx, size(X))
    clfft.set_layout!(p, :interleaved, :interleaved)
    clfft.set_result!(p, :inplace)
    clfft.bake!(p, queue)

    clfft.enqueue_transform(p, :forward, [queue], bufX, nothing)  
    result = cl.read(queue, bufX)

    @assert isapprox(norm(result - fft(X)), zero(Float32))
    
end


myCLFFT (generic function with 1 method)

In [8]:
N = 4096
vector_CLFFT = [1:1:N];
getTime_CLFFT = @time myCLFFT(vector_CLFFT);

LoadError: myCLFFT not defined
while loading In[8], in expression starting on line 56

### 3.2 NFFT

Esse pacote permite a implementação em Julia de uma Não-equidistante Transformada Rápida de Fourier (NFFT). Basicamente é a Transformada Discreta de Fourier com nós de amostrasgem não-equidistantes em qualquer Fourier ou domínio de tempo/espaço. Em contraste com a FFT, a NFFT é um algoritmo aproximado da qual sua precisão pode ser controlada por dois parâmetros: A largura da Janela **_m_** e o fator de subamostragem $\sigma$.

In [9]:
using NFFT 

function myNFFT(x)
    N = length(x)
    x = linspace(-0.4, 0.4, N)      # nodes at which the NFFT is evaluated
    fHat = randn(N)+randn(N)*1im    # data to be transformed
    p = NFFTPlan(x, N)              # create plan. m and sigma are optional parameters
    f = nfft_adjoint(p, fHat)       # calculate adjoint NFFT
    g = nfft(p, f)                  # calculate forward NFFT
end


myNFFT (generic function with 1 method)

In [14]:
N = 4096
vector_NFFT = [1:1:N];
getTime_NFFT = @time myNFFT(vector_NFFT);

elapsed time: 0.015160731 seconds (1563584 bytes allocated)


## 4. Conclusão

A Tabela abaixo fornece a informação do tempo de compilação de cada algoritmo e a sua quantidade de bytes alocados.

In [16]:
using DataFrames

#Necessário o truncamento no @time, DEFEITO!
#Dados Ilustrativos

tabela = DataFrame(Função = ["MyFFT","Native FFT","MyIFFT","Native IFFT","CLFFT","NFFT"],
#Tempo_de_Compilação_segundos = [getTime_myFFT, getTime_ntFFT, getTime_myIFFT, getTime_CLFFT, getTime_NFFT])
Tempo_de_Compilação_segundos = ["----",0.195218919,"----",0.006755123,0.268256433,0.01308109],
#Bytes_Alocados = [getByte_myFFT, getByte_ntFFT, getByte_myIFFT, getByte_CLFFT, getByte_NFFT])
Bytes_Alocados = ["----",6178220,"----",67512,10507482,1563584])

,Função,Tempo_de_Compilação_segundos,Bytes_Alocados
1,MyFFT,----,----
2,Native FFT,0.195218919,6178220
3,MyIFFT,----,----
4,Native IFFT,0.006755123,67512
5,CLFFT,0.268256433,10507482
6,NFFT,0.01308109,1563584


## 5. Referências

Agradecimentos aos colegas de turma : Braga, Johnathan S. ; Souza, Elitelma S.

http://pkg.julialang.org/

https://github.com/tknopp/NFFT.jl

https://github.com/JuliaGPU/CLFFT.jl

https://github.com/FugroRoames/TypedTables.jl